In [8]:
from transformers import Wav2Vec2CTCTokenizer
import pandas as pd
import json

aa = pd.read_csv("../labels.csv")
unit2id = aa[['id','char']].set_index("char").to_dict()['id']

with open('vocab.json', 'w') as f:
    json.dump(unit2id, f)

asfd = Wav2Vec2CTCTokenizer('vocab.json',
                     bos_token = '<sos>',
                     eos_token = '<eos>',
                     pad_token = '<pad>',
                     word_delimiter_token = " ")

In [20]:
import argparse

args = argparse.ArgumentParser()
args.add_argument('--mode', type=str, default='train') 
config = args.parse_args()



usage: ipykernel_launcher.py [-h] [--mode MODE]
ipykernel_launcher.py: error: unrecognized arguments: --ip=127.0.0.1 --stdin=9003 --control=9001 --hb=9000 --Session.signature_scheme="hmac-sha256" --Session.key=b"3d430260-4071-433f-9518-ccea1d8c0441" --shell=9002 --transport="tcp" --iopub=9004 --f=/Users/rainism/Library/Jupyter/runtime/kernel-v2-92974cXu5DwBihFcW.json


SystemExit: 2

/Users/rainism/opt/anaconda3/envs/torch/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3441: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [19]:
import torch

asfd.decode(torch.tensor([3,3,3,3,3,6,8,0,0,0,0,0,3,3,3,5,3,3,0,0,0,0,0,0,0,0,0]))

'이아 그'

In [45]:
def data_processing(data, data_type="train"):
    spectrograms = []
    labels = []
    input_lengths = []
    label_lengths = []
    for (waveform, _, utterance, _, _, _) in data:
        if data_type == 'train':
            spec = train_audio_transforms(waveform).squeeze(0).transpose(0, 1)
        elif data_type == 'valid':
            spec = valid_audio_transforms(waveform).squeeze(0).transpose(0, 1)
        else:
            raise Exception('data_type should be train or valid')
        spectrograms.append(spec)
        label = torch.Tensor(text_transform.text_to_int(utterance.lower()))
        labels.append(label)
        input_lengths.append(spec.shape[0]//2)
        label_lengths.append(len(label))

    spectrograms = nn.utils.rnn.pad_sequence(spectrograms, batch_first=True).unsqueeze(1).transpose(2, 3)
    labels = nn.utils.rnn.pad_sequence(labels, batch_first=True)

    return spectrograms, labels, input_lengths, label_lengths

tensor(2)

In [4]:
seq1 = 100
mel = 80

import torch
spec = torch.randn([mel, seq1]).transpose(0,1)
spec1 = torch.randn([mel, 120]).transpose(0,1)
spectrograms = [spec, spec1]

import torch.nn as nn
nn.utils.rnn.pad_sequence(spectrograms, batch_first=True).unsqueeze(1).transpose(2,3).shape

torch.Size([2, 1, 80, 120])

In [6]:
nn.utils.rnn.pad_sequence(spectrograms, batch_first=True).shape

torch.Size([2, 120, 80])

In [30]:
import torch.nn as nn
import torch
template = nn.ZeroPad2d((0,3-5,0,0))

aa = torch.randn([1,5,5])
template(aa)

tensor([[[ 0.2820, -0.0804,  0.7618],
         [-0.1416,  0.9416, -0.7956],
         [ 0.0633, -0.8903,  1.0790],
         [-0.6923,  1.2591,  0.4260],
         [-0.4973, -0.4562, -0.4418]]])

In [39]:
targets = torch.randint(100, [8,30])

zero_targets = torch.zeros(8, 30).to(torch.long)

for idx, target in enumerate(targets):
    zero_targets[idx].narrow(0,0,len(target)).copy_(torch.LongTensor(target))
    # break


In [40]:
zero_targets

tensor([[52, 68, 13, 25, 82, 32, 22, 19, 49,  8, 56,  9, 32, 25, 28, 54, 70, 11,
         13, 32, 43, 97, 96, 82, 85, 23, 79, 36,  8, 81],
        [47, 82,  5, 61, 18, 67, 55,  8, 40, 66, 20,  9, 76, 53, 67, 55,  9,  6,
         54, 11, 50, 66, 79,  9, 29, 59, 66, 15, 82, 88],
        [16, 36, 11, 28, 37, 33, 45,  3, 35, 80, 31,  8, 38, 36,  2, 74, 71, 37,
         24, 81, 46,  5, 30, 48, 21, 57, 84, 68, 61, 99],
        [37, 22, 58, 45, 47, 60, 91, 41, 97, 28, 89, 44, 30, 96, 12, 63, 93, 10,
         87, 93, 68, 37, 80, 29, 35, 15, 27, 61, 67, 69],
        [46, 88, 58, 53, 57,  9, 56,  8, 95, 30, 41, 70, 79, 18, 24,  4, 54, 72,
         74, 16, 43, 44, 75, 44, 89, 50, 99, 18, 27, 59],
        [ 3, 55,  1,  3, 24, 53, 26,  2,  6, 89, 32, 42, 81, 80, 68, 26, 94, 93,
         56, 38, 66, 17, 42, 32, 48, 84, 89,  9, 48, 79],
        [43, 87, 13, 27, 94, 65, 91, 42, 94, 42, 85, 20, 71, 20, 13, 43,  0, 39,
         30, 45, 60, 23, 52, 78,  9, 91, 72, 95, 56, 11],
        [ 3, 27,  2, 56, 66

In [42]:
targets

tensor([[52, 68, 13, 25, 82, 32, 22, 19, 49,  8, 56,  9, 32, 25, 28, 54, 70, 11,
         13, 32, 43, 97, 96, 82, 85, 23, 79, 36,  8, 81],
        [47, 82,  5, 61, 18, 67, 55,  8, 40, 66, 20,  9, 76, 53, 67, 55,  9,  6,
         54, 11, 50, 66, 79,  9, 29, 59, 66, 15, 82, 88],
        [16, 36, 11, 28, 37, 33, 45,  3, 35, 80, 31,  8, 38, 36,  2, 74, 71, 37,
         24, 81, 46,  5, 30, 48, 21, 57, 84, 68, 61, 99],
        [37, 22, 58, 45, 47, 60, 91, 41, 97, 28, 89, 44, 30, 96, 12, 63, 93, 10,
         87, 93, 68, 37, 80, 29, 35, 15, 27, 61, 67, 69],
        [46, 88, 58, 53, 57,  9, 56,  8, 95, 30, 41, 70, 79, 18, 24,  4, 54, 72,
         74, 16, 43, 44, 75, 44, 89, 50, 99, 18, 27, 59],
        [ 3, 55,  1,  3, 24, 53, 26,  2,  6, 89, 32, 42, 81, 80, 68, 26, 94, 93,
         56, 38, 66, 17, 42, 32, 48, 84, 89,  9, 48, 79],
        [43, 87, 13, 27, 94, 65, 91, 42, 94, 42, 85, 20, 71, 20, 13, 43,  0, 39,
         30, 45, 60, 23, 52, 78,  9, 91, 72, 95, 56, 11],
        [ 3, 27,  2, 56, 66

In [31]:
aa

tensor([[[ 0.2820, -0.0804,  0.7618, -0.5526, -1.1497],
         [-0.1416,  0.9416, -0.7956, -0.7145, -0.3202],
         [ 0.0633, -0.8903,  1.0790, -0.5745, -1.1206],
         [-0.6923,  1.2591,  0.4260, -0.9567, -0.2630],
         [-0.4973, -0.4562, -0.4418, -0.2685,  0.2842]]])

In [26]:
import librosa
import sklearn
import torch
from noisereduce import reduce_noise
import numpy as np

def remove_noise_data(np_wav, ratio=16_000):
    return reduce_noise(y = np_wav, sr=ratio, stationary=False)

def detect_silence(pcm, audio_threshold = 0.0075, min_silence_len = 3, ratio=16000, make_silence_len=1):
    if len(pcm) < min_silence_len*ratio:
        return pcm
    
    b = np.where((abs(pcm) > audio_threshold) == True)[0] # 소리가 나는 부분
    c = np.concatenate(([0], b[:-1]), axis=0)

    starts = c[(b-c)>min_silence_len*ratio]               # 소리가 안나는 부분 시작
    ends = b[(b-c)>min_silence_len*ratio]

    if len(ends) == 0:
        return pcm
    else:
        non_masking = np.array([True]*len(pcm))
        for (s,e) in zip(starts, ends):
            non_masking[s:e+1] = False
            non_masking[e-make_silence_len*ratio:e+1] = True
        
        return pcm[non_masking]


def wav2image_tensor(path):
    audio, sr = librosa.load(path, sr=16_000)
    audio, _ = librosa.effects.trim(audio)

    audio = remove_noise_data(audio)

    if True:
        audio = detect_silence(
            audio,
            audio_threshold=0.0075,
            min_silence_len=3,
            ratio = 16_000,
            make_silence_len=1
            )

    ######### 하드 코딩 된 부분들
    mfcc = librosa.feature.mfcc(
        y = audio, 
        sr=16_000, 
        n_mfcc=80, 
        n_fft=400, 
        hop_length=160
    )

    if True:
        mfcc = remove_noise_data(mfcc)


    ########################3 하드 코딩 부분 변경 ##################
    # max_len = 1000
    mfcc = sklearn.preprocessing.scale(mfcc, axis=1)
    # def pad2d(a, i): return a[:, 0:i] if a.shape[1] > i else np.hstack(
    #     (a, np.zeros((a.shape[0], i-a.shape[1]))))
    # padded_mfcc = pad2d(mfcc, max_len).reshape( 
    #     1, self.config.n_mels, max_len)  # 채널 추가
    mfcc = torch.tensor(mfcc, dtype=torch.float)

    #######################################################   reshape을 해줘야 할 수 도있음.  deepspeech2인가 하는놈은 bs, feat, leng로 들어가는 것 같은데..

    return mfcc

path = '/Users/rainism/Desktop/2023_AI_hub/2023_preliminary_kasr/task2_03.wav'

audio = wav2image_tensor(path)

/Users/rainism/opt/anaconda3/envs/torch/lib/python3.9/site-packages/sklearn/preprocessing/_data.py:240: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn(
/Users/rainism/opt/anaconda3/envs/torch/lib/python3.9/site-packages/sklearn/preprocessing/_data.py:259: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn(


In [27]:
audio.shape

torch.Size([80, 275])

In [9]:
# remove noise
audio.min(), audio.max()

(-0.84535474, 0.885125)

In [21]:
# withoud scale, remove_noise
mfcc.min(),mfcc.max()

(-694.2284, 245.41486)

In [19]:
# with sklearn scale, remove_noise
mfcc.min(),mfcc.max()

(-4.6030335, 5.488033)

In [23]:
mfcc.min(), mfcc.max()

(-611.1071, 210.56094)

In [25]:
mfcc.min(), mfcc.max()

(-4.1439986, 5.7186747)